In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta

Open the file with company names and urls. Name columns appropriately and drop what is not needed.

In [10]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

In [2]:
path = find('AllUrls.csv', 'C://')
urls = pd.read_csv(path)

Drop unnecessary columns and give the remaining columns appropriate names.

In [3]:
urls = urls.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 6', 'Unnamed: 7'])
urls = urls.rename(columns={'Unnamed: 4':'Target URL', 'Unnamed: 8':'Acquiror URL'})

In [4]:
urls.head()

,Date Effective,Target Name,Target URL,Acquiror Name,Acquiror URL
0,29/12/2017 00:00,Netrisk.hu,netrisk.hu,MCI EuroVentures 1 0,https://mci.pl/en
1,26/12/2017 00:00,Health Integrated Inc,healthintegrated.com,ExlService Holdings Inc,https://www.exlservice.com/
2,21/12/2017 00:00,HealthSun Health Plans Inc,healthsun.com,Anthem Inc,anthem.com
3,14/12/2017 00:00,Brighter Inc,https://providers.brighter.com/,Cigna Corp,cigna.com
4,04/12/2017 00:00,EC Insurance Co Ltd,https://www.ecic.co.uk/,Markel Intl Ins Co Ltd,http://www.markelinternational.com/


Fill empty values and errors with 'None'.

In [5]:
urls = urls.fillna(value='None')
urls = urls.replace(to_replace='#ERROR!', value='None')

In [6]:
urls['Date Effective'] = pd.to_datetime(urls['Date Effective'], dayfirst=True)

Make all urls in the form http(s)://.....

In [7]:
for index, row in urls.iterrows():
    if urls.iloc[index]['Target URL'] != 'None':
        if (urls.iloc[index]['Target URL'].startswith('http')==False):
            new_url = 'https://' + urls.iloc[index]['Target URL']
            urls.at[index, 'Target URL'] = new_url
        if ('www' not in '/'.join(urls.iloc[index]['Target URL'].split('/', 2)[-1:])) and (urls.iloc[index]['Target URL'] != 'None'):
            new_url = 'https://www.'+'/'.join(urls.iloc[index]['Target URL'].split('/',2)[2:])
            urls.at[index, 'Target URL'] = new_url
    if urls.iloc[index]['Acquiror URL'] != 'None':
        if (urls.iloc[index]['Acquiror URL'].startswith('http')==False):
            new_url = 'https://' + urls.iloc[index]['Acquiror URL']
            urls.at[index, 'Acquiror URL'] = new_url
        if ('www' not in '/'.join(urls.iloc[index]['Acquiror URL'].split('/', 2)[-1:])):
            new_url = 'https://www.'+'/'.join(urls.iloc[index]['Acquiror URL'].split('/',2)[2:])
            urls.at[index, 'Acquiror URL'] = new_url

Add columns for the URLs before and after the date effective

In [8]:
urls['Target one week prior'] =""
urls['Acquiror one week prior'] =""
urls['Target two years after'] =""
urls['Acquiror two years after'] =""
urls['Target URL one week prior']=""
urls['Acquiror URL one week prior']= ""
urls['Target URL two years after'] = ""
urls['Acquiror URL two years after']= ""

Create a file in which all the URLs before the date effective are stored. This is the URL + the date one week before the date effective.

In [14]:
file = open('C:/URLSBefore.txt', 'w') 

for index, row in urls.iterrows():
    date_effective = urls.iloc[index]['Date Effective'].date()
    date_one_week_prior = date_effective + relativedelta(weeks=-1)
    date_one_week_prior = str(date_one_week_prior).replace('-',"")
    
    if urls.iloc[index]['Target URL'] != 'None':
        target_address_one_week_prior = 'https://web.archive.org/web/' + date_one_week_prior  +'/'+ urls.iloc[index]['Target URL']
        urls.at[index, 'Target one week prior'] = date_one_week_prior
        urls.at[index, 'Target URL one week prior'] = target_address_one_week_prior
        file.write(target_address_one_week_prior+ '\n')
    if urls.iloc[index]['Acquiror URL'] != 'None':
        acquiror_address_one_week_prior = 'https://web.archive.org/web/' + date_one_week_prior  +'/'+ urls.iloc[index]['Acquiror URL'] 
        urls.at[index, 'Acquiror one week prior'] = date_one_week_prior
        urls.at[index, 'Acquiror URL one week prior'] = acquiror_address_one_week_prior
        file.write(acquiror_address_one_week_prior+ '\n')
file.close()

Create a file in which all the URLs after the date effective are stored. This the URL + the date two years after the date effective.

In [15]:
file = open('C:/URLSAfter.txt', 'w') 

for index, row in urls.iterrows():
    date_effective = urls.iloc[index]['Date Effective'].date()
    date_two_years_after = date_effective + relativedelta(years=+2)
    date_two_years_after = str(date_two_years_after).replace('-',"")
   
    if urls.iloc[index]['Target URL'] != 'None':
        target_address_two_years_after = 'https://web.archive.org/web/' + date_two_years_after  +'/'+ urls.iloc[index]['Target URL']
        urls.at[index, 'Target two years after'] = date_two_years_after
        urls.at[index, 'Target URL two years after']  = target_address_two_years_after
        file.write(target_address_two_years_after+ '\n')
    if urls.iloc[index]['Acquiror URL'] != 'None':
        acquiror_address_two_years_after = 'https://web.archive.org/web/' + date_two_years_after  +'/'+ urls.iloc[index]['Acquiror URL'] 
        urls.at[index, 'Acquiror two years after'] = date_two_years_after
        urls.at[index, 'Acquiror URL two years after'] = acquiror_address_two_years_after
        file.write(acquiror_address_two_years_after+ '\n')
file.close()

Convert the date string to a datetime object

In [11]:
urls['Date Effective']=urls['Date Effective'].dt.strftime('%Y-%m-%d')

Save the URL data to Excel

In [12]:
urls.to_excel('C:/CompanyUrls.xlsx')